In [ ]:
https://gradio.app/docs/

In [ ]:
!pip install gradio

In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env', verbose=True)

True

In [2]:
import os

BIGQUERY_PROJECT_ID = os.environ.get('BIGQUERY_PROJECT_ID')
BIGQUERY_DATASET = os.environ.get('BIGQUERY_DATASET')
BIGQUERY_DATASET_REGION = os.environ.get('BIGQUERY_DATASET_REGION')
BIGQUERY_TABLE = os.environ.get('BIGQUERY_TABLE')

VERTEXAI_PROJECT_ID = os.environ.get('VERTEXAI_PROJECT_ID')
VERTEXAI_REGION = os.environ.get('VERTEXAI_REGION')

BUCKET_NAME = os.environ.get('BUCKET_NAME')
BUCKET_URI = os.environ.get('BUCKET_URI')
BUCKET_REGION = os.environ.get('BUCKET_REGION')

PREFIX = os.environ.get('PREFIX')


print("BIGQUERY_PROJECT_ID: ",BIGQUERY_PROJECT_ID)
print("BIGQUERY_DATASET: ",BIGQUERY_DATASET)
print("BIGQUERY_DATASET_REGION: ",BIGQUERY_DATASET_REGION)
print("BIGQUERY_TABLE: ",BIGQUERY_TABLE)

print("VERTEXAI_PROJECT_ID: ",VERTEXAI_PROJECT_ID)
print("VERTEXAI_REGION: ",VERTEXAI_REGION)

print("BUCKET_NAME: ",BUCKET_NAME)
print("BUCKET_URI: ",BUCKET_NAME)
print("BUCKET_REGION: ",VERTEXAI_REGION)


PIPELINE_ROOT = 'gs://{}/pipeline_root'.format(BUCKET_NAME)

BIGQUERY_PROJECT_ID:  datafusionsbox
BIGQUERY_DATASET:  dataset4ccc
BIGQUERY_DATASET_REGION:  us
BIGQUERY_TABLE:  df_for_model_ccc_with_weights
VERTEXAI_PROJECT_ID:  datafusionsbox
VERTEXAI_REGION:  us-central1
BUCKET_NAME:  gcp-demo-ccc-vertexai
BUCKET_URI:  gcp-demo-ccc-vertexai
BUCKET_REGION:  us-central1


In [3]:
table_id = f"{BIGQUERY_PROJECT_ID}.{BIGQUERY_DATASET}.{BIGQUERY_TABLE}"
_user_id = "synerise_client_id"
_target_column="y_if_trans"
_weight_column="weight"
_excluded_columns = [_user_id, _target_column, _weight_column]

In [4]:
from google.cloud import bigquery

client = bigquery.Client()
table = client.get_table(table_id)  # Make an API request.

print(
    "Got table '{}.{}.{}'.".format(table.project, table.dataset_id, table.table_id)
)

Got table 'datafusionsbox.dataset4ccc.df_for_model_ccc_with_weights'.


In [5]:
_features = []
for schema_field in table.schema:
    if schema_field.name not in _excluded_columns:
        _features.append(schema_field.name)

In [6]:
import gradio as gr
import requests

In [7]:
import gradio as gr
import requests
from google.cloud import aiplatform
from google.cloud.aiplatform import Feature, Featurestore

aiplatform.init(
    project=VERTEXAI_PROJECT_ID,
    location=VERTEXAI_REGION
)

endpoint_filter_str='labels.experiment_name="'+f"{PREFIX}-experiments"+'"'

endpoints = aiplatform.Endpoint.list(
       filter=endpoint_filter_str,
)
endpoint = endpoints[0]

####### HERE WE WILL LOOKUP FEATURE STORE
FEATURESTORE_ID = f"{PREFIX}_promotions"
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=VERTEXAI_PROJECT_ID,
    location=VERTEXAI_REGION,
)
print(fs.gca_resource)
stats_entity_type = fs.get_entity_type(entity_type_id="stats")
#########################################

def greet(in_threshold, in_user_id):
    threshold = in_threshold
    user_id = in_user_id
    
    #### Lookup Featurestore for given user_id 
    response = stats_entity_type.read(entity_ids=[user_id], feature_ids=_features)
    instance = response.values.tolist()[0][1:]
    print(instance)
    
    #### Make call to vertex AI Endpoint
    result = endpoint.predict(instances=[instance])
    print(result)
    classification_result = False
    if result.predictions[0][0] > threshold:
        classification_result = True
    return classification_result, result.predictions[0], result.deployed_model_id, result.model_version_id, result.model_resource_name

    
gInputs = [gr.Slider(0, 1, label = 'classification threshold', value = 0.5),
           gr.Textbox(label = 'synerise_client_id', value = '3129545367')
          ]

##Prediction(predictions=[[0.00370752811]], deployed_model_id='2233288435920011264', 
##model_version_id='3', model_resource_name='projects/339239659794/locations/us-central1/models/1047599285782183936', explanations=None
gOutputs = [gr.Textbox(label="Result"),
            gr.Textbox(label="Prediction"),
            gr.Textbox(label="Response: deployed model ID"),
            gr.Textbox(label="Response: model version ID"),
            gr.Textbox(label="Response: model resource name")
           ]

demo = gr.Interface(fn=greet, inputs=gInputs, outputs=gOutputs, allow_flagging='never')

demo.launch(share=True) 

name: "projects/339239659794/locations/us-central1/featurestores/ccc_promotions"
create_time {
  seconds: 1676542011
  nanos: 997796000
}
update_time {
  seconds: 1676542012
  nanos: 528878000
}
etag: "AMEw9yOOnpGLDUSHBRclPqFIV_trZcl7Ff3vum0XUKbaVO0ChUDQ1wZjWSH7NnWsnUoE"
online_serving_config {
  fixed_node_count: 1
}
state: STABLE

Running on local URL:  http://127.0.0.1:7860
Running on public URL: https://5a0821f6-333f-459f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


### Debug

In [8]:
FEATURESTORE_ID = f"{PREFIX}_promotions"
fs = Featurestore(
    featurestore_name=FEATURESTORE_ID,
    project=VERTEXAI_PROJECT_ID,
    location=VERTEXAI_REGION,
)
print(fs.gca_resource)
stats_entity_type = fs.get_entity_type(entity_type_id="stats")

name: "projects/339239659794/locations/us-central1/featurestores/ccc_promotions"
create_time {
  seconds: 1676542011
  nanos: 997796000
}
update_time {
  seconds: 1676542012
  nanos: 528878000
}
etag: "AMEw9yOm8KY8Zw7lXxlQ2dcg-_9C-muoBzQHVoJ1kuwaeSXXjBz9Qlh2CtwTAoK7wPqP"
online_serving_config {
  fixed_node_count: 1
}
state: STABLE



In [21]:
import numpy as np
instances = [['3692184251'], ['2374580415'], ['4549777227'], ['1684193966'], ['4757806082']]
response = np.concatenate(instances)
response

array(['3692184251', '2374580415', '4549777227', '1684193966',
       '4757806082'], dtype='<U10')

In [24]:
import json 
fjson = json.load(open("instances.json"))
fjson

{'instances': [['3692184251'],
  ['2374580415'],
  ['4549777227'],
  ['1684193966'],
  ['4757806082']]}

In [27]:
user_ids = fjson["instances"]
user_ids
print("user ids",user_ids)

user ids [['3692184251'], ['2374580415'], ['4549777227'], ['1684193966'], ['4757806082']]


In [36]:
instance_ids=list(np.concatenate(user_ids))
instance_ids

['3692184251', '2374580415', '4549777227', '1684193966', '4757806082']

In [51]:
instances = stats_entity_type.read(entity_ids=instance_ids, feature_ids = _features)
instances

,entity_id,www_visits_month_freq_1,www_visits_month_freq_2,www_visits_month_freq_3,www_pages_visited_month_1,www_pages_visited_month_2,www_pages_visited_month_3,www_time_onsite_avg_month_1,www_time_onsite_avg_month_2,www_time_onsite_avg_month_3,...,app_add_to_wishlist_month_freq_1,app_add_to_wishlist_month_freq_2,app_add_to_wishlist_month_freq_3,app_search_freq_month_freq_1,app_search_freq_month_freq_2,app_search_freq_month_freq_3,crm_club_level_id,crm_store_purchase_month_freq_1,crm_store_purchase_month_freq_2,crm_store_purchase_month_freq_3
0,1684193966,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,1,0,0
1,2374580415,0,0,0,0,0,0,0.0,0.0,0.0,...,3,0,0,2,0,0,0,0,0,0
2,3692184251,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,1,0,0,0
3,4549777227,0,0,0,0,0,0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,4757806082,0,0,1,0,0,2,0.0,0.0,207.0,...,0,0,0,0,0,0,0,0,0,0


In [55]:
inputs = instances.drop("entity_id", axis=1)
inputs.shape

(5, 67)

In [56]:
res = inputs.values.tolist()

In [57]:
res

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  1.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  1.0,
  0.0,
  0.0,
  4.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  3.0,
  0.0,
  0.0,
  2.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0],
 [0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.